In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline
import os
import sys
import math
import glob
import shutil
import tempfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18}) # defaults to 10.0

In [ ]:
def ffmpeg(basename, outname):
    cmd = "ffmpeg -y -i %s_%%04d.png -vf \"scale=trunc(iw/2)*2:trunc(ih/2)*2\" -pix_fmt yuv420p %s" % (basename, outname)
    os.system(cmd)

In [ ]:
def animate(directory, name):
    tmp = tempfile.mkdtemp()
    files = glob.glob(os.path.join(directory, name) + "_????.csv")
    files.sort()
    print(tmp)

    for file in files:
        base = os.path.basename(file)
        
        df = pd.read_csv(file)
        fig = plt.figure(figsize=(12,9))
        ax = fig.gca()
        x = df['x']
        ax.plot(x,df['theta'], 'r-',linewidth=3.0, label=r'$\theta$')
        ax.plot(x,df['Upsilon'], 'b-',linewidth=3.0, label=r'$\Upsilon$')
        plt.title(name)
        ax.legend(loc='center right')
        ax.set_xlabel('Position')
        plt.savefig(os.path.join(tmp, base[:-3] + "png"), dpi=150, bbox_inches='tight')
        plt.close('all')

    # make video
    ffmpeg(os.path.join(tmp, name), name + ".mp4")

    # cleanup
    shutil.rmtree(tmp)

# Steady State Interface Verification

We first run the steady state input files using `magpie-opt`.

In [ ]:
if os.system("../../../magpie-opt -i steady_theta.i") or os.system("../../../magpie-opt -i steady_upsilon.i"):
    print("Magpie run failed.")    
else:
    print("Success.")

Next we generate two movies showing how the linear ramp converges to teh analytical steady state solution.

In [ ]:
def steady_anim(var, T, ycols):
    tmp = tempfile.mkdtemp()
    #patterns = ["out/steady_theta_2.0_2.0_432_sol_%04d.csv", "out/steady_theta_2.0_2.0_432_ana_%04d.csv"]
    #patterns = ["out/steady_upsilon_2.0_2.0_532.14_sol_%04d.csv", "out/steady_upsilon_2.0_2.0_532.14_ana_%04d.csv"]
    #patterns = ["out/steady_%s_2.0_2.0_%s_sol_%04d.csv", "out/steady_%s_2.0_2.0_%s_ana_%04d.csv"]
    patterns = ["out/steady_%s_2.2_1_%s_sol_%04d.csv", "out/steady_%s_2.2_1_%s_ana_%04d.csv"]


    xcols = ['x', 'x']
    labels = ['numerical', 'analytical']
    line = ['-r', '--b']

    png = "out_" + var + "_%04d.png"
    mp4 = "steady_" + var + ".mp4"
    n = 1

    while True:
        d = []
        for p in patterns:
            file = p % (var, T, n)
            print(file)
            if not os.path.isfile(file):
                break
            # load data
            d.append(pd.read_csv(file))
        else:
            # do all work with loaded data
            print("render")
            n += 1

            fig = plt.figure(figsize=(12,9))
            ax = fig.gca()
            for i in range(len(d)):
                ax.plot(d[i][xcols[i]], d[i][ycols[i]], line[i],linewidth=3.0, label=labels[i])

            #         plt.title(name)
            ax.legend(loc='center right')
            ax.set_xlabel('Position')
            
            outfile = os.path.join(tmp, png % n)
            plt.savefig(outfile, dpi=150, bbox_inches='tight')
            plt.close('all')

            continue
        break

    # make video
    cmd = "ffmpeg -y -i %s -vf \"scale=trunc(iw/2)*2:trunc(ih/2)*2\" -pix_fmt yuv420p %s" % (os.path.join(tmp, png), mp4)
    os.system(cmd)

    # cleanup
    shutil.rmtree(tmp)
    
steady_anim('theta', '432', ['theta', 'theta'])
steady_anim('upsilon', '532.14', ['Upsilon', 'Upsilon'])

# Interface Velocity Verification

We first run the velocity input files using `magpie-opt`.

In [ ]:
for var in ['upsilon', 'theta']:
    for T in range(392, 542,10):
        print(var, T)
        os.system("../../../magpie-opt -i velocity_%s.i Temp_=%d KE=1.933" % (var, T))

Next we collect the PostProcessor CSV files and extract the numerical interface velocity as `dpos/dt` and the analytically computed interface velocity `vanalytic`. The latter is computed in the input uisng a Function and a FunctionValuePostprocessor. The value is constant over time.

In [ ]:
files1 = glob.glob("out/velocity_theta_1.933_1_???.csv")
files1.sort()

x1 = []
y1 = []
ay1 = []
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    x1.append(T)
    y1.append(np.average(list(d['dpos'][-20:]/d['dt'][-20:]))*1e-9)
    ay1.append(np.average(list(d['vanalytic'][-20:]))*1e-9)

m1,b1 = np.polyfit(x1, ay1, 1) 

files2 = glob.glob("out/velocity_upsilon_1.933_1_???.csv")
files2.sort()

x2 = []
y2 = []
ay2 = []
for f in files2:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    x2.append(T)
    y2.append(np.average(list(d['dpos'][-100:]/d['dt'][-100:]))*1e-9)
    ay2.append(np.average(list(d['vanalytic'][-100:]))*1e-9)

m2,b2 = np.polyfit(x2, ay2, 1) 

fig = plt.figure(figsize=(12,9))
ax = fig.gca()

ax.plot(x1, y1, 'o',linewidth=3.0, label=r'$v_\vartheta$ numerical')
ax.plot(x1, m1*np.array(x1)+b1, '-', label=r'$v_\vartheta$ analytical')

ax.plot(x2, y2, 'o',linewidth=3.0, label=r'$v_\Upsilon$ numerical')
ax.plot(x2, m2*np.array(x2)+b2, '-', label=r'$v_\Upsilon$ analytical')

ax.legend(loc='lower right')
ax.set_xlabel('Temperature [K]')
ax.set_ylabel('Velocity [m/s]')

plt.savefig('velocity.png', dpi=150, bbox_inches='tight')
#plt.close('all')

Here we take a look at the interface velocity data from every timestep in the `theta` and `Upsilon` runs. We need to make sure that the velocity converges to a constant value.

In [ ]:
files1 = glob.glob("out/velocity_theta_1.933_1_???.csv")
files1.sort()

fig = plt.figure(figsize=(12,9))
ax = fig.gca()
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    ax.plot(d['dpos']/d['dt'], '*',label="T=%d" % T)

ax.legend(loc='upper right')
ax.set_xlabel('step')
ax.set_ylabel('Velocity [m/s]')


In [ ]:
files1 = glob.glob("out/velocity_upsilon_1.933_1_???.csv")
files1.sort()

fig = plt.figure(figsize=(12,9))
ax = fig.gca()
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    ax.plot(d['dpos']/d['dt'], '*',label="T=%d" % T)

ax.legend(loc='lower right')
ax.set_xlabel('step')
ax.set_ylabel('Velocity [m/s]')


Here we look at the `theta` and `Upsilon` interface profiles to verify that we have the expected clean interfaces. Change the temperature value `T` to look at different runs.

In [ ]:
T = 392
files1 = glob.glob("out/velocity_theta_1.933_1_%d_op_*.csv" % T)
files1.sort()

fig = plt.figure(figsize=(22,9))
ax = fig.gca()
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    ax.plot(d['x'], d['theta'], 'r')
    ax.plot(d['x'], d['Upsilon'], 'b')


In [ ]:
T = 392
files1 = glob.glob("out/velocity_upsilon_1.933_1_%d_op_*0.csv" % T)
files1.sort()

fig = plt.figure(figsize=(22,9))
ax = fig.gca()
for f in files1:
    d = pd.read_csv(f)
    T = float(f.split('_')[-1].split('.')[0])
    ax.plot(d['x'], d['theta'], 'r')
    ax.plot(d['x'], d['Upsilon'], 'b')
